# **HW9: Paragraph Segmentation**

Welcome to the final homework of the semester! We introduce the task of Paragraph Segmentation, where given several sentences of documents, our goal is to predict if a sentence marks the beginning of a new paragraph or not in the document. 

Our aim is to help you better understand the importance of feature extraction and how to curate linguistic features yourselves in training NLP models. We suggest a few methods for you to implement (Part 1), and ask you later to improve upon them using your own ideas (Part 2).







# **Part 1** (70 points)

###**Dataset**
You will be provided with a dataset called '*Paraseg*' consisting of sentences from different documents, divided into train, dev and test sets. Each sentence is followed by a tab and a label either 'B', or 'I'. A new line denotes the end of a document.

'B' means that the current sentence is the beginning of a new paragraph, and in all other cases, the label is 'I'.


Note: One assumption we make in model training is we treat each sentence separately and not belonging to any particular document. Maybe you can improve upon this later?

First things first, let's mount the dataset to Google Drive. You may upload the handout folder 'HW9' to the drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
origin_path = "/content/drive/MyDrive/Colab Notebooks/NLP HW9/paraseg/"

train_path = origin_path + "train/paraseg.train" # set path to train file  "/content/drive/MyDrive/HW9/paraseg/train/paraseg.train" 
dev_path =  origin_path + "dev/paraseg.dev" # set path to dev file  "/content/drive/MyDrive/HW9/paraseg/dev/paraseg.dev"
test_path = origin_path + "test/paraseg.test" # set path to test file "/content/drive/MyDrive/HW9/paraseg/test/paraseg.test" 

As in previous assignments, we'll be working with PyTorch

In [3]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 24.2 MB/s 


In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

import torchmetrics


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Please ensure you run on GPU

In [5]:
device

device(type='cuda')

## **Data Preprocessing and Word Embeddings**
Our first goal is to preprocess the data. Furthermore, we want to obtain embeddings for each sentence. 

In order to do that, we use a simple technique of mapping each word to an index. Using an Embedding layer, we can obtain a feature embedding of each word in a sentence. This was very similar to what you did in the previous homeworks, so we won't ask you to implement this again. 



In [6]:
OOV_token = 0  # We consider out-of-vocabulary tokens (OOV)

In [7]:
class WordToIndex:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "OOV"} 
        self.n_words = 1 
        self.n_sentences = 0

    def addSentence(self, sentence):
        self.n_sentences += 1
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def indexesFromSentence(self, sentence):
        return [self.word2index.get(word, OOV_token) for word in sentence.split(' ')]

    def tensorFromSentence(self, sentence):
        indexes = self.indexesFromSentence(sentence)
        indexes = torch.tensor(indexes, dtype=torch.long)
        return indexes

In [8]:
# common words such as 'the', 'a', 'an', etc. are called stopwords
# our objective is to remove them since they contribute less to model prediction
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def prepareData(path, word_to_index, train):
    f = open(path, 'r') 
    docs = []
    doc = []
    x = f.readlines()
    for row in x:
        if row == '\n':
            docs.append(doc)
            doc = []
            if(len(docs)) == 1000 and train==False:
                break   # We consider only 1000 documents for dev, test for resource purposes
            if(len(docs)) == 5000: #We consider only 5000 documents for training for resource purposes
                break
        else:
            [sentence, label] = row.split('\t')
            sentence = re.sub(r'[^\w\s]', '', sentence.lower())  #lower case and remove punctuations
            filtered_words = [word for word in sentence.split() if word not in stopwords.words('english')]
            sentence = " ".join(filtered_words)

            doc.append((sentence.strip(),label.strip()))
            if train:
                word_to_index.addSentence(sentence)
    return docs

In [11]:
#  word_to_index will help you convert each sentence to a tensor of word indices
# [train/dev/test]_docs is a list of documents. Each document in turn is a list of (sentence, label) tuples
word_to_index = WordToIndex()
train_docs = prepareData(train_path, word_to_index, train = True)
dev_docs = prepareData(dev_path, word_to_index, train = False)
test_docs = prepareData(test_path, word_to_index, train = False)

While you wait for the code to prepare the data (and this may time around 10-12 minutes), let's think about this task for a bit. 

We are finding word embeddings for each sentence for now. But later, we want to define an **embedding for an entire sentence** to help us predict its label. If we have word embeddings for each word in a sentence, what ways can we use it to get an entire sentence embedding? 

In [12]:
test_docs

In [ ]:
# define the number of features you want for word embeddings and define an embedding layer. 
num_embeddings = 50 #you may want to play around with this number and note how it affects model performance
embed_layer = nn.Embedding(word_to_index.n_words, num_embeddings) # embedding layer, input is vocabulary size and output is embedding size

In [ ]:
# this cell is for your understanding of how to obtain a sentence embedding
sentence_word_indices = word_to_index.tensorFromSentence("hello this just for testing")
sentence_word_embedding = embed_layer(sentence_word_indices) # embed each word

print(sentence_word_embedding.shape) # (num_words, embedding_size)
sentence_embedding = sentence_word_embedding.flatten() # flatten each word embedding to obtain a sentence embedding
print(sentence_embedding.shape) # we now have a 1d tensor representing each sentence.

#will flattening work for sentences of different lengths? may need to pad
#other ways to obtain a sentence embedding could be averaging or summing up each word embedding

## **Feature Extraction**

We see a simple way to obtain a sentence embedding of an entire sentence using its word embeddings. We can simply pass these embeddings to our training model to obtain a prediction: 1 (indicating para break 'B') or 0 (otherwise)

However, as always, we can do better. We can capture better features from the sentences and use that to predict our labels. The next sections describe few methods to extract such linguistic features, and your task will be to implement them


###1. Discourse Markers (10 points)

Words such as 'however', 'although', 'because', etc. can be good indicators of a sentence at the beginning of a paragraph. 

We ask you to curate a list of such words. Say your list was ['word1', 'word2'] then the presence/absence of word1 in the sentence: "word1 repeats word1 times not word2" can result in a one-hot-encoded vector [1, 0]

You may also choose to instead store the count of the words as a vector, in this case [2,0]

This embedding may later be concatenated with the sentence embedding to provide more information about the label

In [ ]:
discourse_list = #TODO: define discourse list []

In [ ]:
# sentence is a string, discourse_list is a list of strings
def discourse_markers(sentence, discourse_list):
    in_features = # TODO: this should be the length of the discourse list 
    out_features = # TODO: define number of output features
    markers = #TODO: initialize as list of 0s of size input_features

    sentence = sentence.split()
    for word in sentence:
        for j, marker in enumerate(discourse_list):
            if word == marker:
                #TODO: store presence or absence of word or its count in sentence

    markers = # TODO: convert list to a tensor
    return markers

In [ ]:
markers = discourse_markers("however i have to also say however again", ["however", "but", "also"]).tolist()
assert markers == [1,0,1] or markers == [2,0,1]

###2. Cosine Similarity (10 points)

If a sentence is different, in context, from its previous sentence then there is a high chance of it being the beginning of a new paragraph. 

We can leverage this idea to compute the cosine similarity between the sentence embedding of the current sentence with its previous (or previous two sentences?, up to you). There are probably much better ways to capture this idea, but we stick with a simple implementation. 

In [ ]:
cos = # define torch cosine similarity function, be sure to define dim as 0

def cosine_sim(sentence_1_embed, sentence_2_embed): # sentence_1 and sentence_2 are embeddings of shape (embedding_size)
    cos_sim = # TODO: find cosine similarity between two input sentences
    #print the shape of this tensor. Since this is just one item, we may need to reshape this tensor

    cos_sim = #TODO: reshape the tensor to size 1
    return cos_sim

In [ ]:
s1 = embed_layer(word_to_index.tensorFromSentence("hey are you watching the soccer world cup")).flatten()
s2 = embed_layer(word_to_index.tensorFromSentence("say soccer one more time i dare you")).flatten()

cos_sim = cosine_sim(s1, s2)
assert cos_sim.shape == torch.Size([1])

cos_sim = cosine_sim(torch.Tensor([0.2147, -1.911, 3.456]), torch.Tensor([0.4444, -0.6782, 1.121]))
assert round(cos_sim.item(), 4) == 0.9623

### 3. Co-reference Chains (20 points)

Co-reference resolution refers to the task of identifying words/phrases which refer to the same entity. We can use this important feature in our task. 

Say we are currently trying to predict the label for sentence *s{i}* , we can look at a window of previous sentences say *s{i-2}, s{i-1}* and the next sentences *s{i+1}, s{i+2}*. If a coreference exists between any one of the previous sentences *s{i-2}* or  *s{i-1}* and the current or next sentences *s{i}*, *s{i+1}, or s{i+2}* then we can say that the current sentence *s{i}* should NOT be a new paragraph (since it deals with a topic discussed in the preceding and subsequent sentences).

We can use [fastcoref](https://pypi.org/project/fastcoref/) for this. This takes as input a single string and predicts the co-reference clusters in them. Please refer to the documentation carefully.

Using the function get_clusters() we can obtain the different clusters.

In [ ]:
!pip install fastcoref

In [ ]:
from fastcoref import FCoref
coref_model = FCoref(device=device)

In [ ]:
def get_clusters(combined_sent): #combined sent is a string, consisting of the previous, current and next sentences
    preds = #TODO: call predict function of coref_model and pass the sentence as specified in the documentation (list of a single concatenated sentence)
    clusters = #TODO: obtain the coref cluster indices (Note: these indices are character indices)
    return clusters

In [ ]:
def num_corefs(prev_sentences, cur_sentence, next_sentences):
    combined_sent = ""
    len_prev_sentences = # you may use this variable to check if a point in a cluster belongs in the prev_sentences or not

    # TODO: combine the previous, current and next sentences and store as a single string 'combined_sent'
    
    clusters = #TODO: obtain clusers from the combined sentence

    count = 0

    #TODO: based on the clusters, count the number of coreferences that exist from previous sentences to current OR the next sentences
    # such a coref will exist if a cluster consists of at least one point from prev_sentences and another point from cur_sentence OR next_sentences

    count = #convert to tensor of size 1
    return count
    pass

In [ ]:
sentence1 = 'roger novak and rafa are best friends'
sentence2 = 'roger is swiss'
sentence3 = 'rafa is left handed'

ret = num_corefs([sentence1], sentence2, [sentence3])
assert ret.shape == torch.Size([1])
assert ret.item() == 2

### 4. Lines since last segment (5 points)

A very important feature is checking which line number the current sentence is in the current paragraph. If the sentence is the first or second line in a paragraph, then there is low chance of a paragraph break. But if the line was, say, the 10th line in a para, then the intuition is it has a higher chance of being a paragraph segment. 

You need to make some simple changes in the code below for that. Please refer to variable 'para_len'

### Preparing sentence embeddings (10 points)

So far, we've processed our data and implemented some feature extraction methods. 

Now, we tie everything up. We obtain sentence embeddings for each sentence (using word embeddings and the methods describes above). We will then pass these modified and improved sentence embeddings to our training model

In [ ]:
# this function allows us to obtain a sentence embedding from all the word (embeddings)
# apart from flattening, you can average or sum out word embeddings

def pool_word_embeddings(sent_embedding):
    #input dimensions: (num_words, embedding_size)
    #output dimensions: (embedding_size)

    sent_embedding = #TODO: we expect you to sum each word embedding to obtain a sentence embedding
    return sent_embedding

In [ ]:
se = torch.tensor([[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]])
se = pool_word_embeddings(se)
assert torch.equal(se, torch.tensor([0.2,0.4,0.6]))

In [ ]:
def get_sentence_embeddings(docs):
    sentence_embeddings = []
    labels = []
    prev = None
    para_len = 0    # 4. a very important feature 

    for i, doc in enumerate(docs):
        for j,row in enumerate(doc):
            sentence = row[0] 
            label = #TODO: 'B' indicates 1, otherwise 0
            labels.append(label)

            if row[1] == 'B':
                # set para_len to 0
            else:
                # increment para_len
            
            para_len = # TODO: convert to tensor of shape 1

            markers = #TODO: obtain discourse markers using the current sentence and the discourse list you defined
            
            sent_tensor = word_to_index.tensorFromSentence(sentence)
            sent_embedding = embed_layer(sent_tensor) 

            sent_embedding = #TODO: pool word embeddings to obtain sentence embedding

            cos_sim = torch.FloatTensor([0])
            if prev!=None:
                cos_sim = #TODO: find cosine similarity between current and previous sentence
            prev = sent_embedding 

            #uncomment the lines below to calculate number of corefs
            # num_coref = torch.FloatTensor([0])
            # if j>0 and j<len(doc)-1:
            #     prev_sentences = [doc[j-1][0]] 
            #     next_sentences = [doc[j+1][0]]
            #     num_coref = num_corefs(prev_sentences, sentence, next_sentences)

            sent_embedding = #TODO: concatenate the sentence embedding with the features obtained from discourse markers, cosine similarity, coreference and para_length
            
            sentence_embeddings.append(sent_embedding)

    sentence_embeddings  = pad_sequence(sentence_embeddings, batch_first=True, padding_value=0) #pad the embeddings
    labels = torch.tensor(labels, dtype=torch.float)
    return sentence_embeddings, labels

In [ ]:
train_sentence_embeddings, train_labels = get_sentence_embeddings(train_docs)
train_labels = train_labels.reshape(train_labels.shape[0], 1)
print(train_sentence_embeddings.shape)
print(train_labels.shape)
assert len(train_sentence_embeddings.shape) == 2

In [ ]:
dev_sentence_embeddings, dev_labels = get_sentence_embeddings(dev_docs)
test_sentence_embeddings, test_labels = get_sentence_embeddings(test_docs)

### Model Creation and Training (15 points)

Great job! We've got some cool sentence embeddings now, which can definitely do a good job at our task of predicting paragraph segmentations. 

Let us now implement a simple MLP model for our training. We can instead use various sophisticated models like an LSTM too. 

In [ ]:
# you are free to change the architecture of this MLP
# you may add more layers to increase complexity, or change the hidden size
class MLP(nn.Module):
    def __init__(self, input_size, output_size = 1): #output size is 1 for binary classification
        super(MLP,self).__init__()
        hidden_size = #TODO: define size of hidden layer
        self.linear1 = # TODO: define hidden linear layer
        self.relu1 = # TODO: define ReLU activation layer
        self.linear2 = # TODO: define output layer 
        self.sigmoid = # TODO: define a sigmoid layer
         
    def forward(self, input):
        # implement the forward function of the MLP
        pass

In [ ]:
# Now we train

# we define our optimizer (Adam) and loss function as binary cross entropy loss
mlp = #TODO: define training model, input size is the number of features of our modified sentence embeddings
learning_rate = # TODO: define a suitable learning rate, we recommend trying out with 1e-5 first
optimizer = optim.Adam(mlp.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()
mlp = mlp.to(device) #shift model to cuda
loss_fn = loss_fn.to(device)

num_epochs = 5 # you may experiment with different num_epochs
num_sentences = train_sentence_embeddings.shape[0]
batch_size = 10 # you may define another batch size
mlp.train()

for i in range(num_epochs):
    for j in range (0, num_sentences, batch_size):
        x_train = train_sentence_embeddings[j: j+batch_size] 
        y_train = torch.tensor(train_labels[j : j+batch_size])  
        x_train = x_train.clone().detach()
        y_train = y_train.clone().detach()

        x_train = x_train.to(device)
        y_train = y_train.to(device)

        op = mlp(x_train)

        optimizer.zero_grad()
        loss = loss_fn(op, y_train)
        loss.backward() 
        optimizer.step()
    
    if i%1 == 0:
        print("epoch {} loss : {}".format(i,loss.item()))
    

<ipython-input-150-9781b75c511f>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(train_labels[j : j+batch_size])


epoch 0 loss : 0.10408451408147812
epoch 1 loss : 0.07082154601812363
epoch 2 loss : 0.017085690051317215
epoch 3 loss : 0.012588994577527046
epoch 4 loss : 0.004164569079875946


In [ ]:
def eval(sentence_embeddings, labels): #evaluation function
    hypothesis = []
    reference = []
    mlp.eval()
    for i in range (0, num_sentences, batch_size):
        with torch.no_grad():
            x_dev = sentence_embeddings[i: i+batch_size]  
            y_dev = torch.tensor(labels[i: i+batch_size])  
            x_dev = x_dev.clone().detach()
            y_dev = y_dev.clone().detach()

            x_dev = x_dev.to(device)
            y_dev = y_dev.to(device)

            op = mlp(x_dev)
            y_pred = torch.where(op >= 0.5, 1.0, 0.0)  # simple idea for binary classification ; if probability >=0.5 classify as 1 else as 0
            
            hypothesis.extend(y_pred.squeeze().tolist())
            reference.extend(y_dev.squeeze().tolist())
    
    return hypothesis, reference

### WinDiff Metric

Although we can calculate accuracy to measure how well our predictions match the labels, another very important metric for segmentation tasks is the WinDiff metric. You can learn more about it from this [link](https://www.nltk.org/_modules/nltk/metrics/windowdiff.html)

The WinDiff metric will calculate an absolute score, and we use a window size of 3 for our evaluation of this task. Do note, lower the WinDiff score, the better the model.

In [ ]:
def winDiff(hypothesis, reference, k):
    if len(hypothesis) != len(reference):
        raise ValueError("Segmentations have unequal length")
    wd = 0
    for i in range(len(hypothesis) - k):
        wd += abs(hypothesis[i:i+k+1].count(1) - reference[i:i+k+1].count(1))
    return wd

In [ ]:
hypothesis, reference = eval(dev_sentence_embeddings, dev_labels)
wd = winDiff(hypothesis, reference, 3)

In [ ]:
print('WindowDiff:', wd)
print(hypothesis)
print(reference)

# Part 2 (30 points)

### 2.1 What else can you try? (20 points)

Our aim this whole assignment was not to burden you with trying to improve a baseline, but rather focus your learning on extracting different features and understand why these features are important in improving model performance. 

We gave you 4 simple ideas to extract features and a simple MLP model. But there are several limitations in them. Can you try improving this?

In this section, we ask you to try **at least two new experiments** with your own ideas for feature extraction or improving the model. You can come up with a new feature altogether, or make some significant changes to the model. Tabulate your scores using these new ideas, and mention them in a report. In the report, we also expect a short description for each of your ideas and why you think it improved or did not improve the results. 

We can help you with some ideas:


> 1. Let's think about the cosine similarity metric. Is it really helpful in capturing the similarity between sentences? Maybe not in this case where we use word indices to obtain word embeddings. Can we use pre-trained word embeddings like [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)?
> 2. We use a simple MLP model. You can add more layers to this model. Or you can even try using an LSTM.
> 3. Capturing coreference considers only the immediate previous and next sentence. You can try improving this window. Also do you think passing whole numbers to the model works better, or do you want to try normalizing them?

You've learnt a great deal about NLP in this course; use this part of the Homework to be creative and show-off your newly developed NLP skills. **We'd highly appreciate and reward new & innovative ideas!**

**Note**: changes in the hyperparameters alone, such as changing the learning rate or embedding size etc. only, will not be accepted as an experiment. We want your idea to be concrete (and you can specify what hyperparamaters you used).

Need help? Feel free to reach out during Office Hours or on Piazza, we will be happy to discuss ideas.



### 2.2 Predictions on Test set (10 points)


Choose your best model and sentence embedding technique, by looking at results obtained on the val set, and use it to obtain predictions on the test set. 

The following code should help store hypothesis on the test set to a .txt file as needed in the submission. We will evaluate the WinDiff score obtained by your best model on a threshold set by us. 

Be sure to use the best model and concatenate the best sentence embedding features in test_sentence_embeddings, and update the hypothesis variable



In [ ]:
hypothesis, actuals = eval(test_sentence_embeddings, test_labels)

In [ ]:
with open("/content/drive/MyDrive/HW9/predictions.txt", "w") as output:
    output.write("\n".join(str(item) for item in hypothesis))

# **Submission**

You are expected to submit:

1. **para_seg.py**: copy the required functions or classes from the notebook to the python file given in the handout. We require only the functions and classes mentioned. 
2. **predictions.txt**: the output predictions file generated on the hidden test case
3. **report.pdf**: in a separate written submission, submit your report as specified in Part 2 (2.1). Describe your experiments i.e. why you think it should improve performance, your implementation, and discuss your results